In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
import requests
import zipfile, io

url = 'https://www.dropbox.com/s/g0l60pmw6xet2cu/train.zip?dl=1'

request = requests.get(url, stream=True)

In [3]:
zip_filename = os.path.basename(url)[:-5]
file_dir = './.temp/'
with open(file_dir + zip_filename, 'wb') as zfile:
    zfile.write(request.content)


In [4]:
with zipfile.ZipFile(file_dir + zip_filename, "r") as z:
    z.extractall(path= file_dir)

In [5]:
df = pd.read_csv(file_dir + 'train.csv', sep=',', iterator=True, chunksize=20000000, low_memory = False)
df = pd.concat(df, ignore_index=True)

In [6]:
df.sample(5)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
8276222,8276222,2013-07-13,51,358519,6.0,NaN
74058884,74058884,2016-03-22,18,1457417,1.0,False
111342427,111342427,2017-04-04,2,409739,1.0,False
25818851,25818851,2014-06-23,2,1239815,5.0,False
3903001,3903001,2013-04-05,47,268664,40.0,NaN


In [7]:
df  = df[df.item_nbr==103501 ].groupby("date")['unit_sales'].sum().reset_index()
df.to_csv(file_dir + 'train_item_103501.csv')

In [8]:
df

,date,unit_sales
0,2013-01-02,185.0
1,2013-01-03,153.0
2,2013-01-04,155.0
3,2013-01-05,160.0
4,2013-01-06,173.0
...,...,...
1620,2017-08-11,80.0
1621,2017-08-12,84.0
1622,2017-08-13,103.0
1623,2017-08-14,88.0


In [ ]:
train = pd.read_csv("train.zip")

In [ ]:
top1 = train[train.item_nbr == 265559 ]
top1['date'] = pd.to_datetime(top1['date'])
top1['year'] = top1['date'].dt.year

#### Подготовим данные

In [ ]:
unit_sales_by_date = top1.groupby('date').sum()['unit_sales']

#### Импортируем метрики

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

### Тест Адфуллера для проверки стационарных временных рядов

In [ ]:
from statsmodels.tsa.stattools import adfuller

X = unit_sales_by_date.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

if result[0] < result[4]["5%"]:
    print ("Reject Ho - Time Series is Stationary")
else:
    print ("Failed to Reject Ho - Time Series is Non-Stationary")

## Модель скользящего среднего

In [ ]:
def moving_average_forecast(series, window_size):
    forecast = []
    for time in range(len(series) - window_size):
        forecast.append(series[time:time + window_size].mean())
    return np.array(forecast)
moving_average_days = 6
moving_avg = moving_average_forecast(unit_sales_by_date,moving_average_days )# 

print(moving_avg.shape,unit_sales_by_date.shape)

print("mean_squared_error",mean_squared_error(unit_sales_by_date.values[moving_average_days:], moving_avg))
print("mean_absolute_error",mean_absolute_error(unit_sales_by_date.values[moving_average_days:], moving_avg))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(unit_sales_by_date.values[moving_average_days:], moving_avg))


plt.figure(figsize=(15,6))

plt.plot(unit_sales_by_date.values[moving_average_days:], label="Actual")
plt.plot(moving_avg, label="Moving average")
plt.ylabel("Total Unit Sold")
plt.xlabel("Day")
plt.title("Moving Average Forecast")
plt.legend(loc="upper right")

In [ ]:
x_train = unit_sales_by_date['2013':'2016'].values
x_test  = unit_sales_by_date['2017'].values

df = pd.DataFrame()

df["Original Values"]  = unit_sales_by_date
df["shift1"] = df["Original Values"].shift()
df["shift2"] = df["shift1"].shift()
df["shift3"] = df["shift2"].shift()
df["shift4"] = df["shift3"].shift()
df["shift5"] = df["shift4"].shift()
df["shift6"] = df["shift5"].shift()


lag_value = 6
df.dropna(inplace=True)


x_train, y_train = df['2013':'2016'].iloc[:,0:lag_value].values, df['2013':'2016'].iloc[:,lag_value:].values
x_test, y_test = df['2017'].iloc[:,0:lag_value].values, df['2017'].iloc[:,lag_value:].values

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train, y_train)
print(reg.coef_)
print(reg.intercept_)


ar_predictions = reg.predict(x_test)

plt.figure(figsize=(15,6))
plt.plot(ar_predictions ,label = "Predictions")
plt.plot(y_test, label = "Original" )
plt.xlabel("Days")
plt.ylabel("Sale Count")

print("mean_squared_error",mean_squared_error(y_test, ar_predictions))
print("mean_absolute_error",mean_absolute_error(y_test, ar_predictions))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, ar_predictions))


plt.legend(loc="upper right")

Итак, чтобы построить модель нам нужно знать ее порядок, состоящий из 3-х параметров:
    
    p — порядок компоненты AR
    d — порядок интегрированного ряда
    q — порядок компонетны MA
    
d мы уже знаем - это 1

осталось определить p и q. Для их определения нам надо изучить авторкорреляционную(ACF) и частично автокорреляционную(PACF) функции для ряда первых разностей.

ACF поможет нам определить q, т. к. по ее коррелограмме можно определить количество автокорреляционных коэффициентов сильно отличных от 0 в модели MA
PACF поможет нам определить p, т. к. по ее коррелограмме можно определить максимальный номер коэффициента сильно отличный от 0 в модели AR.

Чтобы построить соответствующие коррелограммы, в пакете statsmodels имеются следующие функции: plot_acf() и plot_pacf(). Они выводят графики ACF и PACF, у которых по оси X откладываются номера лагов, а по оси Y значения соответствующих функций. Нужно отметить, что количество лагов в функциях и определяет число значимых коэффициентов.

In [ ]:
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(unit_sales_by_date.values.squeeze(), lags=6, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(unit_sales_by_date, lags=6, ax=ax2)

После изучения коррелограммы PACF можно сделать вывод, что p = 6, т.к. на ней все лаги сильно отличны от нуля.
По коррелограмме ACF можно предположить, что q = 6, т.к. на лаг 6 значении функций резко возрастает. Итак, когда известны все параметры можно построить модель.

## ARMA Model

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from datetime import datetime, timedelta

x_train, y_train = df['2013':'2016'].iloc[:,0:lag_value], df['2013':'2016'].iloc[:,lag_value:]
x_test, y_test = df['2017'].iloc[:,0:lag_value], df['2017'].iloc[:,lag_value:]

ARMA_model = ARMA(x_train.values.reshape(-1).tolist(), (6,6)).fit()
arma_predictions = ARMA_model.predict(start=len(x_train), end=len(x_train) + len(x_test)-1, dynamic=False)


plt.figure(figsize=(15,6))
plt.plot(arma_predictions ,label = "Predictions")
plt.plot(y_test.values, label = "Original" )
plt.xlabel("Days")
plt.ylabel("Sale Count")
plt.legend(loc="upper right")

print("mean_squared_error",mean_squared_error(y_test, arma_predictions))
print("mean_absolute_error",mean_absolute_error(y_test, arma_predictions))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, arma_predictions))



## ARIMA model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(x_train['Original Values'].values, (6,1,6)).fit()



In [ ]:
fig, ax = plt.subplots(figsize=(17,7))
shown_train_size = 150
train_size = len(x_train['Original Values'].values)
test_size = len(x_test['Original Values'].values) 
ax = x_test.set_index(pd.Series(range(shown_train_size, shown_train_size + test_size)))['Original Values'].plot(ax=ax)
model.plot_predict(start=train_size-shown_train_size,end=train_size+test_size -1,dynamic=False, plot_insample=True,ax=ax)

In [ ]:
arima_predictions = model.predict(train_size, train_size + test_size -1)
print("mean_squared_error",mean_squared_error(y_test, arima_predictions))
print("mean_absolute_error",mean_absolute_error(y_test, arima_predictions))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, arima_predictions))

